In [1]:
import pymongo
from superduperdb import superduper


In [2]:
# setup SuperDuperDB

db = pymongo.MongoClient().documents
db = superduper(db)



INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [3]:
from sklearn.datasets import load_iris
from superduperdb.core.document import Document
from superduperdb.datalayer.mongodb.query import Collection

X, y = load_iris(return_X_y=True)



In [9]:
from sklearn.datasets import load_iris
from superduperdb.core.document import Document
from superduperdb.datalayer.mongodb.query import Collection
from superduperdb.encoders.numpy.array import array


# Load the [iris dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset)
X, y = load_iris(return_X_y=True)
encoder = array(dtype='float64', shape=(4,))

# Create a MongoDB document for each row in the dataset
collection = Collection(name='iris')
data = [Document({'x': encoder(X_value), 'y':int(y_value) }) for (X_value, y_value) in zip(X,y)]

db.execute(
    collection.insert_many(data, encoders=[encoder], valid_prob=0.1)
)


TypeError: Collection.insert_many() got an unexpected keyword argument 'valid_prob'

In [6]:
db.execute(collection.find_one())

Document({'_id': ObjectId('64bfd4bc3d2999403a987663'), 'x': Encodable(x=array([5.1, 3.5, 1.4, 0.2]), encoder=Encoder(identifier='numpy.float64[4]', decoder=<Artifact artifact=<superduperdb.encoders.numpy.array.DecodeArray object at 0x2ad136e00> serializer=dill>, encoder=<Artifact artifact=<superduperdb.encoders.numpy.array.EncodeArray object at 0x2ad136ce0> serializer=dill>, shape=[4], version=1)), 'y': 0, '_fold': 'train'})

In [7]:
from sklearn.linear_model import LogisticRegression
from superduperdb.core.dataset import Dataset
from superduperdb.core.metric import Metric

superduper_clf = superduper(LogisticRegression())


superduper_clf.fit(
    X='x',
    y='y',
    db=db,
    select=collection.find(),
    validation_sets=[
        Dataset(
            identifier='my-eval',
            select=collection.find({'_fold': 'valid'}),
        )
    ],
    
    metrics=[Metric(
        identifier='acc',
        object=lambda x, y: sum([xx == yy for xx, yy in zip(x, y)]) / len(x)
    ),]
)



100%|██████████| 141/141 [00:00<00:00, 219605.22it/s]


LogisticRegression()

In [8]:
superduper_clf.metric_values

{'my-eval/acc': [1.0]}

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# create a pipeline object
pipe = make_pipeline(StandardScaler(),
     LogisticRegression()
 )
sd_pipe = superduper(pipe)

In [9]:
sd_pipe.fit(
    X='x',
    y='y',
    db=db,
    select=Collection(name='iris', ).find(),
)


100%|██████████| 146/146 [00:00<00:00, 240144.46it/s]


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# create a pipeline object
pipe = make_pipeline(StandardScaler(),
     LogisticRegression()
 )
# load the iris dataset and split it into train and test sets
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# fit the whole pipeline
pipe.fit(X_train, y_train)
# we can now use it like any other estimator
accuracy_score(pipe.predict(X_test), y_test)

0.9736842105263158